In [6]:
import matplotlib.pyplot as plt
from scipy.io import wavfile
import argparse
import os
from glob import glob
import numpy as np
import pandas as pd
from librosa.core import resample, to_mono
from tqdm import tqdm


Se leen los archivos .wav de las carpetas 'Archivos_wav' y 'Archivos_wav_test'. Se realiza un downsampling a 16 Khz, y se divide cada muestra a fin de obtener muestras de 2 segundos. Las muestras procesadas se guardan en 'Archivos_procesados' y 'Archivos_procesados_test'.

In [7]:
def downsampling(path, sr):
    rate, wav = wavfile.read(path)
    wav = wav.astype(np.float32, order='F')
    try:
        tmp = wav.shape[1]
        wav = to_mono(wav.T)
    except:
        pass
    wav = resample(wav, rate, sr)
    wav = wav.astype(np.int16)
    return sr, wav

In [8]:
def save_sample(sample, rate, target_dir, fn, ix):
    fn = fn.split('.wav')[0]
    dst_path = os.path.join(target_dir.split('.')[0], fn+'_{}.wav'.format(str(ix)))
    if os.path.exists(dst_path):
        return
    wavfile.write(dst_path, rate, sample)

In [12]:
src_root = 'Archivos_wav'
src_root_test = 'Archivos_wav_test'
dst_root = 'Archivos_procesados'
dst_root_test = 'Archivos_procesados_test'
dt = 2

wav_paths = glob('{}/**'.format(src_root), recursive=True)
wav_paths = [x for x in wav_paths if '.wav' in x]

wav_paths_test = glob('{}/**'.format(src_root_test), recursive=True)
wav_paths_test = [x for x in wav_paths_test if '.wav' in x]


dirs = os.listdir(src_root)
dirs_test = os.listdir(src_root)

os.mkdir(dst_root)
os.mkdir(dst_root_test)

clases = os.listdir(src_root)

In [13]:
for i in clases:  
        target_dir = os.path.join(dst_root, i)    
        target_dir_test = os.path.join(dst_root_test, i)
        os.mkdir(target_dir)  
        os.mkdir(target_dir_test)
        src_dir = os.path.join(src_root, i) 
        src_dir_test = os.path.join(src_root_test, i)
        for fn in tqdm(os.listdir(src_dir)):   
            src_fn = os.path.join(src_dir, fn) 
            rate, wav = downsampling(src_fn, args.sr)  
            delta_sample = int(dt*rate)    
            if wav.shape[0] < delta_sample: 
                sample = np.zeros(shape=(delta_sample,), dtype=np.int16) 
                sample[:wav.shape[0]] = wav  
                save_sample(sample, rate, target_dir, fn, 0) 
            
            else: 
                trunc = wav.shape[0] % delta_sample 
                for cnt, i in enumerate(np.arange(0, wav.shape[0]-trunc, delta_sample)):
                    start = int(i)
                    stop = int(i + delta_sample)
                    sample = wav[start:stop]
                    save_sample(sample, rate, target_dir, fn, cnt)
                    
        for fn in tqdm(os.listdir(src_dir_test)):   
            src_fn = os.path.join(src_dir_test, fn) 
            rate, wav = downsampling(src_fn, args.sr)  
            delta_sample = int(dt*rate)   

            if wav.shape[0] < delta_sample: 
                sample = np.zeros(shape=(delta_sample,), dtype=np.int16) 
                sample[:wav.shape[0]] = wav 
                save_sample(sample, rate, target_dir_test, fn, 0) 
            else: 
                trunc = wav.shape[0] % delta_sample 
                for cnt, i in enumerate(np.arange(0, wav.shape[0]-trunc, delta_sample)):
                    start = int(i)
                    stop = int(i + delta_sample)
                    sample = wav[start:stop]
                    save_sample(sample, rate, target_dir_test, fn, cnt)



100%|██████████| 1/1 [00:02<00:00,  2.00s/it]
